# Power Outages
* **See the main project notebook for instructions to be sure you satisfy the rubric!**
* See Project 03 for information on the dataset.
* A few example prediction questions to pursue are listed below. However, don't limit yourself to them!
    * Predict the severity (number of customers, duration, or demand loss) of a major power outage.
    * Predict the cause of a major power outage.
    * Predict the number and/or severity of major power outages in the year 2020.
    * Predict the electricity consumption of an area.

Be careful to justify what information you would know at the "time of prediction" and train your model using only those features.

# Summary of Findings


### Introduction
This project aims to explore how selected features from the outages dataset can be used to predict the duration of a power outage by training a Random Forest Regression Model. Both RMSE and R-squared will be used to analyze the accuracy and predictive characteristics of the model. First a Baseline model was constructed to serve as a lower bound of what can be expected and was then modified to include transformed features and different parameters. 

### Baseline Model
The Baseline Model was constructed to use 9 features from the outages dataset to form the probablistic trees of the Random Forest. The Month, State, NERC Region, Climate Region, Year, and Cause Category features, being categorical, were one-hot encoded. The Climate Category feature was ordinally encoded (due to Climate Category being ordinal as hot > normal > cold). For the baseline model, the max depth parameter of 5 for the Random Forest Regressor was arbritrarily selected. The baseline model resulted in a model which was overfitted to it training data because its training R2 of ~0.669 was much higher than its test R2 of 0.138. Additionally, the baseline model testing RMSE was 233% higher than its training RMSE- ~6927.913 and 2966.882, respectively. The baseline model has poor predictive performance outside of its training set as well even having relatively poor predictive performance within it. The baseline model has huge room for improvement as it is neither good at interpolation on its training data nor at extrapolating to test data it has not been trained on.

### Final Model
To improve the baseline model, all numerical features ('PC.REALGSP.STATE', 'POPULATION') were standardized to eliminate scaling issues and a new feature was introduced in the form of the hour of the outage start as a transformation of the outage start feature from the original dataset. Outage start hour provides the Random Forest much more resolution on discriminating between outages which occured in the same month year and even day for example. Additionally, the hour feature makes intuitive sense as an outage that started at 3 AM is most likely going to take longer to fix compared to one at 3 PM for example. Next, the best Random Forest hyperparameters were found using a 5-fold grid search cross validation to be max depth of 11, use all features, and 128 trees. In hindsight, the Random Forest model is not well suited to this task as the final model also performed worse on the training dataset in both RMSE and R2. However, the final model was better at its regression task compared to the baseline. The most stark difference is the difference in R2 across the final and baseline model. In the training set, the final model 20% better at 0.818. While the final model's test R2 is still low at around 0.23, it is around 70% better at representing the variance in outage duration correctly. The improvements in RMSE are not as dramatic meaning the final model is still overfitting to the noise in the training set. 

### Fairness Evaluation
I decided to determine if my final model explained the variation in outage duration better for outages affecting higher populations compared to outages in lower population. This is due to the fact there are many more outages present above a population value of 1,000,000 compared to below 1,000,000 and this could be providing the model more clarity and resolution for these more populous areas. To test this I used the difference in R2 between the two subsets (1M> and 1M<) with the null hypothesis positing no significance difference between the R2 of the two sets and the alternative being the R2 for higher populated outages being higher than lower populated areas. The permutation test resulted in a p-value of 0.527, meaning the observed difference in R2 was not statistically significant to reject the null hypothesis. The model does not explain the variance in outage durations better for more populated areas compared to lower populated areas.

# Code

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

In [2]:
#Dataset Import + Cleaning
outage = pd.read_excel(
                io='data/outage.xlsx',
                index_col=0,
                header=5,
                usecols=list(range(1,57)),
                skiprows=[6]
                )

#Replacing NA with NAN
outage.replace(to_replace='NA',value = np.NaN, inplace=True)

#Converting outage start and restoration date/time columns into single column
out_st_dt=outage['OUTAGE.START.DATE'].astype(np.datetime64)
out_st_tm=outage['OUTAGE.START.TIME'].astype(str).astype(np.datetime64).dt.time
out_st = (
    out_st_dt.astype(str)+' '+out_st_tm.astype(str)
        ).replace('NaT NaT',np.NaN).astype(np.datetime64)

rst_dt=outage['OUTAGE.RESTORATION.DATE'].astype(np.datetime64)
rst_tm=\
    outage['OUTAGE.RESTORATION.TIME'].astype(str).astype(np.datetime64).dt.time
rst =  (
        rst_dt.astype(str)+' '+rst_tm.astype(str)
        ).replace('NaT NaT',np.NaN).astype(np.datetime64)

outage.insert(8, 'OUTAGE.START', out_st)
outage.insert(9, 'OUTAGE.RESTORATION', rst)

#Primary features selected for analysis

keeps = [
        'YEAR', 
        'MONTH',
        'POSTAL.CODE',
        'CAUSE.CATEGORY',
        'OUTAGE.START',
        'OUTAGE.RESTORATION',
        'NERC.REGION',
        'CLIMATE.REGION',
        'ANOMALY.LEVEL',
        'CLIMATE.CATEGORY',
        'OUTAGE.DURATION',
        'DEMAND.LOSS.MW',
        'CUSTOMERS.AFFECTED',
        'PC.REALGSP.STATE',
        'PC.REALGSP.REL',
        'POPULATION'
        ]

drops = [col for col in outage.columns if col not in keeps]

#Redundant Features
drops+= [
        'OUTAGE.START.DATE', 
        'OUTAGE.START.TIME', 
        'OUTAGE.RESTORATION.DATE', 
        'OUTAGE.RESTORATION.TIME',
        'DEMAND.LOSS.MW',
        'CUSTOMERS.AFFECTED'
        ]

outage.drop(columns=drops, inplace=True)

outage.head()

,YEAR,MONTH,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START,OUTAGE.RESTORATION,CAUSE.CATEGORY,OUTAGE.DURATION,PC.REALGSP.STATE,PC.REALGSP.REL,POPULATION
OBS,,,,,,,,,,,,,,
1,2011,7.0,MN,MRO,East North Central,-0.3,normal,2011-07-01 17:00:00,2011-07-03 20:00:00,severe weather,3060.0,51268,1.077376,5348119
2,2014,5.0,MN,MRO,East North Central,-0.1,normal,2014-05-11 18:38:00,2014-05-11 18:39:00,intentional attack,1.0,53499,1.089792,5457125
3,2010,10.0,MN,MRO,East North Central,-1.5,cold,2010-10-26 20:00:00,2010-10-28 22:00:00,severe weather,3000.0,50447,1.066826,5310903
4,2012,6.0,MN,MRO,East North Central,-0.1,normal,2012-06-19 04:30:00,2012-06-20 23:00:00,severe weather,2550.0,51598,1.071476,5380443
5,2015,7.0,MN,MRO,East North Central,1.2,warm,2015-07-18 02:00:00,2015-07-19 07:00:00,severe weather,1740.0,54431,1.092027,5489594


### Baseline Model

In [3]:
#Imports and Functions
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (OneHotEncoder, 
                                StandardScaler,
                                OrdinalEncoder, 
                                FunctionTransformer)
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
import math

def rmse(true, pred):
    return math.sqrt(mean_squared_error(true,pred))

In [4]:
states = list(
    outage['POSTAL.CODE']
    .value_counts()[outage['POSTAL.CODE']
    .value_counts()>2].index
)

In [5]:
out = 'OUTAGE.DURATION'
outage = outage.dropna(axis=0, subset=['CLIMATE.REGION', out], how= 'any')
outage = outage[outage['POSTAL.CODE'].isin(states)]
X = outage.drop(out, axis=1)
y = outage[out]

X_train, X_test, y_train, y_test = train_test_split(
                                    X, 
                                    y, 
                                    test_size=0.3,
                                    stratify=outage['POSTAL.CODE'])

In [6]:
rfr = RandomForestRegressor(max_depth=5)

oneh = OneHotEncoder(drop='first')
label = OrdinalEncoder()
si = SimpleImputer(strategy='mean')
keep = FunctionTransformer(lambda x: x)

oneh_col = [
        'MONTH',
        'POSTAL.CODE', 
        'NERC.REGION', 
        'CLIMATE.REGION', 
        'CAUSE.CATEGORY',
        'YEAR',]

label_col = ['CLIMATE.CATEGORY']

keep_num_col = [
        'PC.REALGSP.STATE',
        'POPULATION',]

num_pl = Pipeline([('meanimputer', si),('keep', keep)])


preproc = ColumnTransformer(
    transformers= [
        ("oneh", oneh, oneh_col),
        ("label", label, label_col),
        ("num_conv", num_pl, keep_num_col),
        ]
)

pl = Pipeline(
    [('preproc', preproc),
     ('rfr', rfr),
    ]
)

fit_pl = pl.fit(X_train, y_train)

preds_train  = pl.predict(X_train)
train_r2 = fit_pl.score(X_train, y_train)
train_rmse = rmse(y_train, preds_train)

preds_test = pl.predict(X_test)
test_r2 = fit_pl.score(X_test, y_test)
test_rmse = rmse(y_test, preds_test)

print(f"Training R2 = {train_r2}\nTraining RMSE = {train_rmse}")
print(f"Testing R2 = {test_r2}\nTesting RMSE = {test_rmse}")

Training R2 = 0.6695842824172711
Training RMSE = 2966.8828143744004
Testing R2 = 0.13886042639504137
Testing RMSE = 6927.916224970826


### Final Model

In [12]:
rfr = RandomForestRegressor(max_depth=11, max_features=None, n_estimators=128)

def to_hour(df):
        return(df["OUTAGE.START"].dt.hour).to_frame()

oneh = OneHotEncoder(drop='first')
label = OrdinalEncoder()
si = SimpleImputer(strategy='mean')
std_scl =StandardScaler()
hour = FunctionTransformer(to_hour)



oneh_col = [
        'MONTH',
        'POSTAL.CODE', 
        'NERC.REGION',  
        'CLIMATE.REGION',
        'CAUSE.CATEGORY',
        'YEAR',
                ]

label_col = ['CLIMATE.CATEGORY']

keep_num_col = ['PC.REALGSP.STATE',
        'POPULATION',
        'ANOMALY.LEVEL'
        ]

hour_col = ['OUTAGE.START']

num_pl = Pipeline([('imputer', si),('stdscaler', std_scl)])


preproc = ColumnTransformer(
    transformers= [
        ("oneh", oneh, oneh_col),
        ("label", label, label_col),
        ("num_conv", num_pl, keep_num_col),
        ('hour', hour, hour_col)
        ]
)

pl2 = Pipeline(
    [('preproc', preproc),
     ('rfr', rfr),
    ]
)

fit_pl2 = pl2.fit(X_train, y_train)

preds_train  = pl2.predict(X_train)
train_r2 = fit_pl2.score(X_train, y_train)
train_rmse = rmse(y_train, preds_train)

preds_test = pl2.predict(X_test)
test_r2 = fit_pl2.score(X_test, y_test)
test_rmse = rmse(y_test, preds_test)

print(f"Training R2 = {train_r2}\nTraining RMSE = {train_rmse}")
print(f"Testing R2 = {test_r2}\nTesting RMSE = {test_rmse}")

Training R2 = 0.8183281832524472
Training RMSE = 2199.9551319303478
Testing R2 = 0.23207664841381814
Testing RMSE = 6542.214671158717


In [8]:
def hyperparameter_finder(pl, outage):
    train, test = train_test_split(outage, test_size=0.3)
    pl_fitted = pl.fit(train.drop('OUTAGE.DURATION', axis=1), 
                        train['OUTAGE.DURATION'])
    params = {'classifier__max_depth':range(1,20),
                'max_features': ['auto', 'sqrt', 'log2'],
                'n_estimators':[64,80,96,112,128,144,160]}
    grids = GridSearchCV(pl_fitted[-1], 
                    params=params, 
                    cv=5, 
                    n_jobs=-1)
    grids.fit(train.drop('OUTAGE.DURATION', axis=1), 
                        train['OUTAGE.DURATION'])
    return grids.best_params_, grids.best_score_

#hyperparameter_finder(fit_pl2, outage)
#suppressing function because it took 3 hours to run, best params in final model and best score was ~0.258

### Fairness Evaluation

In [15]:
pop_divider = 1000000.0

pop_lt = outage[outage['POPULATION']<= pop_divider]
pop_gt = outage[outage['POPULATION']> pop_divider]

X1 = pop_lt.drop(out, axis=1)
y1 = pop_lt[out]

X2 = pop_gt.drop(out, axis=1)
y2 = pop_gt[out]

obs_test_stat = (pl2.score(X2, y2)-
                pl2.score(X1, y1)
                )
obs_test_stat


-0.020769214766163224

In [16]:
def trial(outage):
    pop_divider = 1000000.0

    pop_lt = outage[outage['POPULATION']<= pop_divider]
    pop_gt = outage[outage['POPULATION']> pop_divider]

    X1 = pop_lt.drop(out, axis=1)
    y1 = pop_lt[out]

    X2 = pop_gt.drop(out, axis=1)
    y2 = pop_gt[out]

    test_stat = (pl2.score(X2, y2)-
                pl2.score(X1, y1)
                )
    return test_stat

trial(outage)

-0.020769214766163224

In [17]:
#null: r2(lowerhalf) ~= r2(upperhalf)
#alt: r2(lowerhalf) lower than r2(upperhalf)

trials = []
for i in range(1000):
    shuffled= outage.copy()
    shuffled['POPULATION'] = np.random.permutation(shuffled['POPULATION'])
    t = trial(shuffled)
    trials.append(t)

trials = np.array(trials)

trials_df = pd.DataFrame(trials)

p = np.count_nonzero(trials>obs_test_stat)/len(trials)
p

0.527